In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import JavascriptException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import time

import time,json, dataclasses
from typing import List, Optional
import ulid, re, time
from datetime import datetime as dt, timedelta

In [2]:
website_url = "https://www.fincaraiz.com.co/"

In [55]:
def fermer_fenetre_modal(driver, timeout=5):
    try:
        # Attendre que le bouton de fermeture soit cliquable
        close_button = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.close-button"))
        )
        close_button.click()
        print("✅ Modale fermée avec succès.")
    except Exception as e:
        print(f"❌ Impossible de fermer la modale :\n{e}")

def nuke_fincaraiz_cookie_popup(driver, timeout=5, retry_interval=0.5):
    """Supprime le pop-up cookie de Fincaraiz en cliquant sur 'Accept all' ou via JS fallback."""

    print("⏳ Tentative de suppression du popup Fincaraiz...")
    end_time = time.time() + timeout
    popup_gone = False

    while time.time() < end_time:
        try:
            # Tentative de clic sur le bouton 'Accept all' si présent
            buttons = driver.find_elements(By.CSS_SELECTOR, ".fc-button-label")
            for btn in buttons:
                if "accept all" in btn.text.lower():
                    try:
                        btn.click()
                        print("✅ Bouton 'Accept all' cliqué.")
                        popup_gone = True
                        break
                    except (ElementClickInterceptedException, Exception) as e:
                        print(f"⚠️ Erreur lors du clic sur le bouton 'Accept all' : {e}")
            
            if popup_gone:
                break

            # Fallback JS si le bouton n'est pas interactable ou pas encore là
            driver.execute_script("""
                const popup = document.querySelector('.fc-dialog-scrollable-content');
                if (popup) popup.remove();

                const overlay = document.querySelector('.fc-consent-root');
                if (overlay) overlay.remove();

                document.body.style.overflow = 'auto';
            """)
            still_there = driver.execute_script("""
                return document.querySelector('.fc-dialog-scrollable-content') !== null ||
                       document.querySelector('.fc-consent-root') !== null;
            """)
            if not still_there:
                popup_gone = True
                break

        except JavascriptException as e:
            print(f"⚠️ Erreur JS : {e}")

        time.sleep(retry_interval)

    if popup_gone:
        print("🧨 Pop-up Fincaraiz supprimé.")
    else:
        print("❌ Pop-up non supprimé après le délai imparti.")

def clear_ant_select_choices(driver):
    try:
        # Cliquer sur le champ pour forcer l'ouverture du dropdown
        select_area = driver.find_element(By.CSS_SELECTOR, ".ant-select-selector")
        select_area.click()
        time.sleep(0.5)  # laisser le temps à l'UI

        # Ensuite, supprimer les éléments sélectionnés
        remove_buttons = driver.find_elements(By.CSS_SELECTOR, ".ant-select-selection-item-remove")
        for btn in remove_buttons:
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)

        print("✅ Sélections supprimées.")
    except Exception as e:
        print(f"❌ Erreur lors du nettoyage des sélections : {e}")

def back_to_home(driver):
    # set url to https://www.fincaraiz.com.co/
    driver.get(website_url)
    print("Retour à la page d'accueil...")


In [3]:
def create_driver():
    options = Options()
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(2) 
    return driver
driver = create_driver()

In [56]:
fincaraiz_creds_path = r"C:\Users\f.gionnane\Documents\Selenium\fincaraiz Search\finca_raiz_creds.json"
with open(fincaraiz_creds_path, 'r') as file:
    content = json.load(file)
    email = content['email']
    password = content['password']

In [ ]:
def login(driver, email, password):
    # Cliquer sur le bouton de login
    try:
        driver.get(website_url)
        nuke_fincaraiz_cookie_popup(driver)
      
        time.sleep(3)

        # Attendre que le bouton devienne cliquable (jusqu'à 10 secondes)
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-secondary"))
        )

        login_button.click()
        print("Clicked on Ingresar...")

        time.sleep(1.5)
        email_field = driver.find_element(By.ID, "email")
        email_field.send_keys(email)
        print("Filled email field...")
        email_field.send_keys(Keys.ENTER)
        time.sleep(1)

        contrasena_field = driver.find_element(By.ID, "password")
        contrasena_field.send_keys(password)
        contrasena_field.send_keys(Keys.ENTER)
        time.sleep(2)
        username = driver.find_element(By.XPATH, "//span[@class='username']")
        if username:
            print("Connexion réussie!")
            
    except Exception as e:
        print(f"Erreur lors de la connexion :\n{e}")

login(driver, email, password)

In [ ]:
def search(driver, list_real_estate_type, location):
    try:
        back_to_home(driver)
        nuke_fincaraiz_cookie_popup(driver)
        clear_ant_select_choices(driver)

        type_mapping = {
            "apartamento": "Apartamento",
            "casa": "Casa",
            "apartaestudio": "Apartaestudio",
            "cabana": "Cabaña",
            "casa campestre": "Casa Campestre",
            "lote": "Casa Lote",
            "finca": "Finca",
            "habitacion": "Habitación"
        }

        if list_real_estate_type:
            dropdown_real_estate_type = driver.find_element(By.CSS_SELECTOR, ".ant-select-selector")
            dropdown_real_estate_type.click()
            time.sleep(1)

            for item in list_real_estate_type:
                item_lower = item.lower()
                if item_lower in type_mapping:
                    try:
                        option_title = type_mapping[item_lower]
                        option = driver.find_element(By.CSS_SELECTOR, 
                            f"div[title='{option_title}'] div[class='ant-select-item-option-content']")
                        option.click()
                        time.sleep(0.5)
                    except Exception as e:
                        print(f"Erreur lors du clic sur l'option '{option_title}': {e}")
                else:
                    print(f"Type '{item}' non reconnu.")

        if location:
            ubicacion_input = driver.find_element(By.CSS_SELECTOR, "input[placeholder='🔍︎ Busca por ubicación']")
            ubicacion_input.clear()
            ubicacion_input.send_keys(location)
            ubicacion_input.send_keys(Keys.ENTER)
            time.sleep(1)
            print("Recherche en cours...")

        fermer_fenetre_modal(driver)

        search_button = driver.find_element(By.CSS_SELECTOR, "button.search-button")
        search_button.click()

    except Exception as e:
        print(f"Erreur lors de la recherche :\n{e}")


In [53]:
search(driver, ["apartamento", "casa"], "Santa Marta")

Retour à la page d'accueil...
✅ Aucun pop-up cookie détecté.
✅ Sélections supprimées.
💥 Échec lors de la sélection du type de bien : Message: element click intercepted: Element <div class="ant-select-selector">...</div> is not clickable at point (319, 373). Other element would receive the click: <div class="fc-dialog-content">...</div>
  (Session info: chrome=135.0.7049.96)
Stacktrace:
	GetHandleVerifier [0x00007FF7F1C55355+78597]
	GetHandleVerifier [0x00007FF7F1C553B0+78688]
	(No symbol) [0x00007FF7F1A091AA]
	(No symbol) [0x00007FF7F1A67089]
	(No symbol) [0x00007FF7F1A64A42]
	(No symbol) [0x00007FF7F1A61AE1]
	(No symbol) [0x00007FF7F1A60A42]
	(No symbol) [0x00007FF7F1A52204]
	(No symbol) [0x00007FF7F1A870EA]
	(No symbol) [0x00007FF7F1A51AB6]
	(No symbol) [0x00007FF7F1A87300]
	(No symbol) [0x00007FF7F1AAF2BB]
	(No symbol) [0x00007FF7F1A86EC3]
	(No symbol) [0x00007FF7F1A503F8]
	(No symbol) [0x00007FF7F1A51163]
	GetHandleVerifier [0x00007FF7F1EFEF0D+2870973]
	GetHandleVerifier [0x00007FF